In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [50]:
from custom_nets.resnet import ResNet, train_model, evaluate
import optuna

## Pre-processing

This is all some quite elaborate steps to load in the data. It definetely does not need to be as difficult as this. The main goal is to have a train, val and test set of features and labels. As long as you have that all correctly defined it should all work a-ok.

In [51]:
microbiome = pd.read_csv('../../data/raw/curated_metagenomics/relative_abundance.csv',index_col=0).transpose()
metadata = pd.read_csv('../../data/raw/curated_metagenomics/metadata.csv',index_col='sample_id',low_memory=False)

For this example we will try to classify disease from healthy based on microbiome. Disease is classified as diseased (according to the original data) and BMI<16 | BMI=>30. These are the boundaries of severe underweight and obesity.

In [52]:
#get stool samples
metadata = metadata.loc[metadata.body_site == 'stool',:]

#Add obesity disease tags to disease BMI
to_change = metadata.BMI>=30
metadata.loc[to_change,'disease'] = 'obesity'

to_change = metadata.BMI<16
metadata.loc[to_change,'disease'] = 'severe_underweight'

# Remove all disease NaNs
metadata = metadata.loc[metadata.disease==metadata.disease,:]

#Take only the adults
to_keep = metadata.age_category != 'newborn'
metadata = metadata.loc[to_keep,:]

# Get the overlapping set of samples between metadata and microbiome data
overlapping_samples = list(set(metadata.index) & set(microbiome.index))
microbiome= microbiome.loc[overlapping_samples,:]
metadata = metadata.loc[overlapping_samples,:]


base_metadata = metadata.loc[metadata.study_name != 'HMP_2019_ibdmdb',:]
base_microbiome = microbiome.loc[base_metadata.index,:]

target_metadata = metadata.loc[metadata.study_name == 'HMP_2019_ibdmdb',:]
target_microbiome = microbiome.loc[target_metadata.index,:]

Here the class labels and feature names are defined.

In [53]:
y = np.asarray( base_metadata.disease != 'healthy',dtype=int)
feature_names = microbiome.columns

## Split data

Here we split the data into the train val and test sets. Since the curated set is rather big, we stick to 2000 test samples and 1000 validation samples. This can of course be tuned.

After splitting the data is transformed to tensors which are moved to the device, the train code assumes all tensors and moves are already moved to the GPU, this is beneficial as it speeds up the loading of the data a lot. But, it is something to be mindful about.

In [54]:
X_train, X_val, y_train, y_val = train_test_split(base_microbiome,y, test_size=2000)

#Transfer to tensors and bring to device
X_train = torch.tensor(np.asarray(X_train), dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)

X_val = torch.tensor(np.asarray(X_val), dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)

In [55]:
train_loader = DataLoader( TensorDataset(X_train,y_train), batch_size=256, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val,y_val), batch_size=256, shuffle=True,)

dataloaders = {
    'train' : train_loader,
    'val' : val_loader,
}

dataset_sizes = {
    'train' : len(y_train),
    'val' : len(y_val)
}

## Define the objective function for optuna

This is where we define the objective function to be optimized by optuna. This code is also adapted from the sample example as the ResNet code. The trial variables are all optimized in order to maximize validation AUC.


In [56]:
def objective(trial):
    """Define the objective function"""
    residual_dropout_check = trial.suggest_categorical("residual_dropout_check", [True,False])
    residual_dropout = trial.suggest_float('residual_dropout',0,0.5)

    weight_decay_check = trial.suggest_categorical("weight_decay_check", [True,False])
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)

    lr = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)

    if residual_dropout_check:
        params = {
            'd_numerical': len(feature_names),
            'd': trial.suggest_int('d', 64, 512),
            'd_hidden_factor' : trial.suggest_int('d_hidden_factor', 1, 4),
            'n_layers' : trial.suggest_int('n_layers', 1, 8),
            'hidden_dropout' : trial.suggest_float('hidden_dropout',0,0.5),
            'residual_dropout' : residual_dropout,
            'd_out':1
        }
    else:
        params = {
            'd_numerical': len(feature_names),
            'd': trial.suggest_int('d', 64, 512),
            'd_hidden_factor' : trial.suggest_int('d_hidden_factor', 1, 4),
            'n_layers' : trial.suggest_int('n_layers', 1, 8),
            'hidden_dropout' : trial.suggest_float('hidden_dropout',0,0.5),
            'residual_dropout' : 0,
            'd_out':1
        }
    
    model = ResNet(**params)
    model = model.to(device)
    criterion = torch.nn.BCEWithLogitsLoss()
    if weight_decay_check:
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)


    model,results_dict = train_model(model, dataloaders, criterion, optimizer, dataset_sizes, phases= ['train','val'])

    return results_dict['best_val_auc']

Optimize the parameters with optuna, the more trials the better. This is of course a trade-off with computational time.

In [57]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

[I 2023-05-08 12:16:13,777] A new study created in memory with name: no-name-d8340243-c59f-43ed-a9a6-8c8a3b24da65
 10%|▉         | 98/1000 [00:24<03:48,  3.95it/s]
[I 2023-05-08 12:16:38,596] Trial 0 finished with value: 0.7127220630645752 and parameters: {'residual_dropout_check': True, 'residual_dropout': 0.04175273632842147, 'weight_decay_check': False, 'weight_decay': 0.00019875907965767722, 'learning_rate': 0.0007761607804378421, 'd': 223, 'd_hidden_factor': 4, 'n_layers': 7, 'hidden_dropout': 0.17815968283737493}. Best is trial 0 with value: 0.7127220630645752.


early stopping...


 10%|▉         | 95/1000 [00:17<02:45,  5.47it/s]
[I 2023-05-08 12:16:55,969] Trial 1 finished with value: 0.7261648774147034 and parameters: {'residual_dropout_check': False, 'residual_dropout': 0.03191365987222772, 'weight_decay_check': False, 'weight_decay': 4.432823531787194e-05, 'learning_rate': 0.00012212505001069657, 'd': 438, 'd_hidden_factor': 1, 'n_layers': 4, 'hidden_dropout': 0.18517956292272086}. Best is trial 1 with value: 0.7261648774147034.


early stopping...


  8%|▊         | 77/1000 [00:15<03:10,  4.85it/s]
[I 2023-05-08 12:17:11,852] Trial 2 finished with value: 0.7206567525863647 and parameters: {'residual_dropout_check': False, 'residual_dropout': 0.0932470177459912, 'weight_decay_check': False, 'weight_decay': 3.3199484289991568e-06, 'learning_rate': 0.0002535753813506154, 'd': 458, 'd_hidden_factor': 1, 'n_layers': 5, 'hidden_dropout': 0.03955745677796951}. Best is trial 1 with value: 0.7261648774147034.


...

early stopping...


  8%|▊         | 79/1000 [00:22<04:27,  3.44it/s]
[I 2023-05-08 13:15:32,147] Trial 198 finished with value: 0.7273995876312256 and parameters: {'residual_dropout_check': False, 'residual_dropout': 0.08543908441124179, 'weight_decay_check': False, 'weight_decay': 2.5876150873893474e-06, 'learning_rate': 0.00010110731238980266, 'd': 251, 'd_hidden_factor': 1, 'n_layers': 8, 'hidden_dropout': 0.4504102327369312}. Best is trial 175 with value: 0.7496123909950256.


early stopping...


 10%|█         | 104/1000 [00:28<04:04,  3.66it/s]
[I 2023-05-08 13:16:00,600] Trial 199 finished with value: 0.7290316224098206 and parameters: {'residual_dropout_check': False, 'residual_dropout': 0.04062223757756008, 'weight_decay_check': False, 'weight_decay': 5.509534095655811e-06, 'learning_rate': 0.00012951481498739054, 'd': 331, 'd_hidden_factor': 1, 'n_layers': 8, 'hidden_dropout': 0.392373746388397}. Best is trial 175 with value: 0.7496123909950256.


early stopping...


## Retrain the model with the best params
You might recognize this from the objective function, essentially this is the same code, but it will pick the best params found by optuna.

In [58]:
best_params = study.best_params

params = {
    'd_numerical':len(feature_names),
    'd':best_params['d'],
    'd_hidden_factor' : best_params['d_hidden_factor'],
    'n_layers' : best_params['n_layers'],
    'hidden_dropout' : best_params['hidden_dropout'],
    'residual_dropout' : best_params['residual_dropout'],
    'd_out' : 1,
}

model = ResNet(**params)
model = model.to(device)
criterion = torch.nn.BCEWithLogitsLoss()

if best_params['weight_decay_check']:
    optimizer = torch.optim.AdamW(model.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'])
else:
    optimizer = torch.optim.AdamW(model.parameters(), lr=best_params['learning_rate'])    

model,results_dict = train_model(model, dataloaders, criterion, optimizer, dataset_sizes, phases= ['train','val'])

 13%|█▎        | 128/1000 [00:29<03:18,  4.39it/s]

early stopping...


In [65]:
import pickle
with open('../../data/raw/curated_metagenomics/resnet_params_T2D.pkl', 'wb') as fp:
    pickle.dump(params, fp)

In [66]:
torch.save(model.state_dict(), '../../data/raw/curated_metagenomics/resnet_curated_T2D.pt')